In [1]:
import sys, os
import time
import requests

In [2]:
def read_file(filename, chunk_size=5242880):
    with open(filename, 'rb') as _file:
        while True:
            data = _file.read(chunk_size)
            if not data:
                break
            yield data
            
def check_status(endpoint, response):
    '''
    check status at every 0.5 s.
    '''
    status = response.json()['status']
    while status == 'queued' or status == 'failed' or status == 'processing':
        time.sleep(0.5)
        response = requests.get(endpoint, headers=headers)
        status = response.json()['status']
    else:
        return status, response
 

In [3]:
def speech_to_text(audio_file_path):
    '''
    Given an audio file, the function will return the text.
    '''
    
    upload_url_response = requests.post('https://api.assemblyai.com/v2/upload',
                         headers=headers,
                         data=read_file(audio_file_path))
    
    # Get the transcription result.
    endpoint = "https://api.assemblyai.com/v2/transcript"

    json = {
      "audio_url": upload_url_response.json()['upload_url']
    }
    id_response = requests.post(endpoint, json=json, headers=headers)

    # perhaps standing in the queue
    
    transcription_id  = id_response.json()['id']
    endpoint = "https://api.assemblyai.com/v2/transcript/" + transcription_id
    id_response = requests.get(endpoint, headers=headers)
    status = id_response.json()['status']
    status_result, id_response = check_status(endpoint, id_response)
    
    # if status is completed, fetch the results
    if status_result == 'completed':
        text_result = id_response.json()['text']
        return audio_file_path, text_result
    else:
        return audio_file_path, 'Not Successful.'

In [11]:
path = 'C:/Users/pzin/OneDrive - Simulations Plus, Inc/Documents/HackNC/AI-Counselor-Pet/preprocessed_datasets/ravdess_dataset/speech/Actor_01/'

# files_in_path = os.listdir(path)

filename = "03-01-01-01-01-02-01.wav"

headers = {
    "authorization": "8a79d503a3de4c05a502b94b26545cd9",
    "content-type": "application/json"
}

start_time = time.time()
file, text = speech_to_text(os.path.join(path, filename))
print(text)
print("--- %s seconds ---" % (time.time() - start_time))


Kids are talking by the door.
--- 9.81611680984497 seconds ---
